In [1]:
!pip install ultralytics 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 5.8 MB/s eta 0:00:00


In [2]:
import os
import shutil
import cv2

def apply_clahe_on_lab(image_path, clip_limit=3.0, tile_size=8):
    """
    Áp dụng CLAHE trên kênh L của không gian màu LAB.
    
    Args:
        image_path (str): Đường dẫn đến file ảnh .jpg.
        clip_limit (float): Giới hạn clip cho CLAHE (khuyến nghị 2.0-4.0).
        tile_size (int): Kích thước tile (ví dụ: 8 cho (8,8)).
    
    Returns:
        np.ndarray: Ảnh đã tăng tương phản.
    """
    # Đọc ảnh (BGR)
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"Không đọc được ảnh: {image_path}")
    
    # Chuyển sang LAB
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    
    # Tách kênh
    l, a, b = cv2.split(lab)
    
    # Tạo CLAHE
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=(tile_size, tile_size))
    
    # Áp dụng CLAHE lên kênh L
    l_clahe = clahe.apply(l)
    
    # Ghép lại
    lab_clahe = cv2.merge((l_clahe, a, b))
    
    # Chuyển về BGR
    result = cv2.cvtColor(lab_clahe, cv2.COLOR_LAB2BGR)
    
    return result

def copy_dataset_with_clahe(source_dir, dest_dir, clip_limit=3.0, tile_size=8):
    """
    Sao chép cấu trúc dataset, áp dụng CLAHE cho ảnh .jpg trong images.
    
    Args:
        source_dir (str): Đường dẫn thư mục gốc (PartA).
        dest_dir (str): Đường dẫn thư mục mới.
        clip_limit (float): Giới hạn clip cho CLAHE.
        tile_size (int): Kích thước tile cho CLAHE.
    """
    # Tạo thư mục đích nếu chưa tồn tại
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)
    
    # Duyệt qua các subfolder chính: test, valid, train
    for subfolder in ['test', 'valid', 'train']:
        source_sub = os.path.join(source_dir, subfolder)
        dest_sub = os.path.join(dest_dir, subfolder)
        
        if not os.path.exists(source_sub):
            print(f"Không tìm thấy {source_sub}, bỏ qua.")
            continue
        
        # Tạo subfolder đích
        os.makedirs(dest_sub, exist_ok=True)
        
        # Xử lý folder 'labels'
        source_labels = os.path.join(source_sub, 'labels')
        dest_labels = os.path.join(dest_sub, 'labels')
        if os.path.exists(source_labels):
            # Sao chép toàn bộ labels (files .txt)
            shutil.copytree(source_labels, dest_labels, dirs_exist_ok=True)
        
        # Xử lý folder 'images'
        source_images = os.path.join(source_sub, 'images')
        dest_images = os.path.join(dest_sub, 'images')
        if os.path.exists(source_images):
            os.makedirs(dest_images, exist_ok=True)
            
            # Duyệt qua các file .jpg
            for filename in os.listdir(source_images):
                if filename.lower().endswith('.jpg'):
                    source_img_path = os.path.join(source_images, filename)
                    dest_img_path = os.path.join(dest_images, filename)
                    
                    # Áp dụng CLAHE và lưu
                    enhanced_img = apply_clahe_on_lab(source_img_path, clip_limit, tile_size)
                    cv2.imwrite(dest_img_path, enhanced_img)
                    print(f"Đã xử lý và lưu: {dest_img_path}")
                else:
                    # Sao chép các file khác nếu có (dù yêu cầu chỉ .jpg, nhưng an toàn)
                    shutil.copy(os.path.join(source_images, filename), dest_images)

# Sử dụng: Thay 'ten_folder_moi' bằng tên bạn muốn (ví dụ: 'PartA_Enhanced')
source_dir = r'/kaggle/input/scut-head/PartA'
dest_dir = r'/kaggle/working/PartA_CLAHE'  # Đặt tên folder mới ở đây

copy_dataset_with_clahe(source_dir, dest_dir, clip_limit=2.0, tile_size=16)

Đã xử lý và lưu: /kaggle/working/PartA_CLAHE/test/images/PartA_01420_jpg.rf.d00201a3d5133fdef24c59a23ea662d8.jpg
Đã xử lý và lưu: /kaggle/working/PartA_CLAHE/test/images/PartA_01391_jpg.rf.1f1810ad5451e960d084664ea8289f21.jpg
Đã xử lý và lưu: /kaggle/working/PartA_CLAHE/test/images/PartA_01603_jpg.rf.ad4cd9bb8654e40e3284b54fb7a0e634.jpg
Đã xử lý và lưu: /kaggle/working/PartA_CLAHE/test/images/PartA_00134_jpg.rf.4cc033221318f8aea638fc9a3b8eb5e4.jpg
Đã xử lý và lưu: /kaggle/working/PartA_CLAHE/test/images/PartA_01761_jpg.rf.66fa72617e6b9c321b6af7676e3a0509.jpg
Đã xử lý và lưu: /kaggle/working/PartA_CLAHE/test/images/PartA_00786_jpg.rf.3b4c219bddfda6532e5da0566f48b5d9.jpg
Đã xử lý và lưu: /kaggle/working/PartA_CLAHE/test/images/PartA_00392_jpg.rf.e2ecc2713d62bbb8255fa6d030c2c597.jpg
Đã xử lý và lưu: /kaggle/working/PartA_CLAHE/test/images/PartA_01270_jpg.rf.a35e7a70f76a38eb86da08738ffd77e6.jpg
Đã xử lý và lưu: /kaggle/working/PartA_CLAHE/test/images/PartA_00452_jpg.rf.90f3563819efb2ce90dc

In [4]:
import yaml

# Cấu hình đường dẫn dataset
# Lưu ý: YOLO cần đường dẫn tuyệt đối tới folder 'images'
data_config = {
    'path': '/kaggle/working/PartA_CLAHE', # Root path (optional nhưng tốt để define context)
    'train': '/kaggle/working/PartA_CLAHE/train/images',
    'val':   '/kaggle/working/PartA_CLAHE/valid/images',
    'test':  '/kaggle/working/PartA_CLAHE/test/images',
    
    # Class configuration
    'nc': 1,           # Số lượng class: chỉ có 1 class là "head"
    'names': ['head']  # Tên class
}

# Lưu file yaml
yaml_path = '/kaggle/working/scut_head.yaml'
with open(yaml_path, 'w') as f:
    yaml.dump(data_config, f)

print(f"✅ Đã tạo file config tại: {yaml_path}")
# %cat {yaml_path} # Uncomment để xem nội dung file

✅ Đã tạo file config tại: /kaggle/working/scut_head.yaml


In [7]:
# 3. Train với P2 head
from ultralytics import YOLO

print("🚀 Load pretrained YOLO26x + override với P2 architecture...")
model = YOLO('yolo12x.pt')  # pretrained weights
# model = YOLO(p2_yaml_path).load('yolo26x.pt')  # override arch P2 + map weights (backbone khớp)

model.info()

print("🚀 Bắt đầu Training với P2 head...")
results = model.train(
    data='/kaggle/working/scut_head.yaml',
    epochs=100,
    imgsz=640,                  # Có thể tăng lên 800-896 nếu VRAM cho phép để tận dụng P2
    batch=8,                    # GIẢM xuống 8 hoặc 4 để tránh OOM (P2 + x nặng hơn base)
    patience=20,                # Tăng lên 20 cho an toàn
    optimizer='AdamW',
    lr0=0.001,
    cos_lr=True,
    # Augmentation mạnh hơn cho small/occluded heads
    mosaic=1.0,
    close_mosaic=10,
    mixup=0.2,                  # Tăng nhẹ
    copy_paste=0.4,             # Tăng cho occlusion
    fliplr=0.5,
    scale=0.6,
    rect=False,                   
    project='SCUT_Head_Project',
    name='yolo12x_CLAHE_v0_640',
    exist_ok=True,
    verbose=True,
    amp=True,                   # Mixed precision để tiết kiệm VRAM
    single_cls=True,
    freeze=2,
)

print("✅ Training hoàn tất!")

🚀 Load pretrained YOLO26x + override với P2 architecture...
YOLOv12x summary: 488 layers, 59,216,928 parameters, 0 gradients, 200.3 GFLOPs
🚀 Bắt đầu Training với P2 head...
Ultralytics 8.4.9 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 14913MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.4, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/kaggle/working/scut_head.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=2, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.2, mode=train, model=yolo12x.